<a href="https://colab.research.google.com/github/basturkme/4IER-HMI/blob/PokeD/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GEREKLİ KÜTÜPHANELERİ YÜKLE
import os
import scipy.io
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# --- 1. AYARLAR ---
# Dosyayı sol taraftaki klasöre yüklediğinden emin ol!
dosya_yolu = 'S1_A1_E1.mat'
HEDEF_HAREKET_ID = 1  # İşaret Parmağı
MODEL_ADI = 'emg_model'

print("--- 1. Veri Okuma Başlıyor ---")

# Dosya var mı kontrol et (Çökmemesi için)
if not os.path.exists(dosya_yolu):
    print(f"\n🛑 HATA: '{dosya_yolu}' dosyası bulunamadı!")
    print("Lütfen sol taraftaki dosya menüsünden .mat dosyasını yüklediğine emin ol.")
    # Kodu burada nazikçe durdur
    raise SystemExit("Dosya eksik olduğu için işlem durduruldu.")

# --- 2. VERİ HAZIRLIĞI ---
try:
    mat = scipy.io.loadmat(dosya_yolu)
    df = pd.DataFrame(mat['emg'])
    df['Label'] = mat['restimulus']
except Exception as e:
    print(f"Veri okuma hatası: {e}")
    raise SystemExit

# Veriyi Süzme
df_hareket = df[df['Label'] == HEDEF_HAREKET_ID].copy()
df_dinlenme = df[df['Label'] == 0].copy()

# Undersampling (RAM Şişmesini Önlemek İçin)
if len(df_hareket) > 0:
    # Dinlenme verisini hareket verisi kadar alıyoruz
    df_dinlenme_esit = df_dinlenme.sample(n=len(df_hareket), random_state=42)
    df_filtered = pd.concat([df_hareket, df_dinlenme_esit])
    df_filtered['Label'] = df_filtered['Label'].apply(lambda x: 1 if x == HEDEF_HAREKET_ID else 0)
    # Karıştır
    df_filtered = df_filtered.sample(frac=1, random_state=42).reset_index(drop=True)
else:
    print("Hata: Hedef hareket dosyada yok.")
    raise SystemExit

# Giriş/Çıkış
X = df_filtered[[0, 1, 2, 3]].values
y = df_filtered['Label'].values

# RAM Tasarrufu: Kullanılmayan büyük değişkenleri silelim
del df, df_hareket, df_dinlenme, mat, df_dinlenme_esit
import gc
gc.collect()

# Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizasyon
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# --- 3. MODEL EĞİTİMİ ---
print("\n--- 2. Model Eğitiliyor ---")
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Eğitimi sessiz modda yap (verbose=0) ki tarayıcı kasmasın
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"EĞİTİM BİTTİ! Doğruluk: %{acc*100:.2f}")

# --- 4. TFLITE DÖNÜŞÜMÜ ---
print("\n--- 3. TFLite Dönüşümü ---")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Dosya olarak kaydet
tflite_filename = f'{MODEL_ADI}.tflite'
with open(tflite_filename, 'wb') as f:
    f.write(tflite_model)

print(f"Model boyutu: {len(tflite_model)} bytes")

# --- 5. GÜVENLİ C HEADER OLUŞTURMA (Saf Python) ---
# Linux komutu (xxd) yerine Python ile yazıyoruz, çökme yapmaz.
print("\n--- 4. Header Dosyası Oluşturuluyor ---")

def hex_to_c_array(data, var_name):
    c_str = ''
    # Hex formatına çevir: 0x0a, 0x1b...
    for i, val in enumerate(data):
        c_str += f'0x{val:02x}, '
        if (i + 1) % 12 == 0:
            c_str += '\n  '

    header_content = (
        f'const unsigned char {var_name}[] = {{\n  {c_str[:-2]}\n}};\n'
        f'const unsigned int {var_name}_len = {len(data)};'
    )
    return header_content

header_filename = f'{MODEL_ADI}_data.h'
c_code = hex_to_c_array(tflite_model, f'{MODEL_ADI}_data')

with open(header_filename, 'w') as f:
    f.write(c_code)

print(f"\n✅ BAŞARILI! '{header_filename}' dosyası oluşturuldu.")
print("Dosyalar menüsünden (solda) dosyayı indirip ESP32 projenize ekleyebilirsiniz.")

--- 1. Veri Okuma Başlıyor ---

--- 2. Model Eğitiliyor ---
EĞİTİM BİTTİ! Doğruluk: %88.79

--- 3. TFLite Dönüşümü ---
Saved artifact at '/tmp/tmp3_3llsal'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140501228254800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140501228255952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140501228254608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140501228241360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140501228256528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140501228252304: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model boyutu: 3040 bytes

--- 4. Header Dosyası Oluşturuluyor ---

✅ BAŞARILI! 'emg_model_data.h' dosyası oluşturuldu.
Dosyalar menüsünden (solda) dosyayı indirip ESP32 projenize ekleyebi